In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import time
from fake_useragent import UserAgent 
ua = UserAgent()
headers = {'user-agent': ua.random}
import scipy import stats

#data = requests.get(url, headers=headers)

#### Get Monthly Sales for 2019 & 2020

In [2]:
def get_monthly_sales_df(url, year_str, tbl_num):
    '''
    Return a time series df of monthly US car sales for every make and model 
    '''
    response = requests.get(url, headers=headers)
    page= response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find car sales data and turn it into a list
    tbl_str = soup.find_all('tbody')[tbl_num].text
    tbl_list = tbl_str.split('\n')
    tbl_list = [i for i in tbl_list if i] 
    
    #create empty time series df for specified year
    date_time_str = year_str + '-01'
    start_date = datetime.datetime.strptime(date_time_str, '%Y-%m')
    index = pd.date_range(start_date, periods=12, freq='m')
    df = pd.DataFrame(index=index)
    
    #fill empty df with monthly sales for all makes and models
    col_name = ''
    idx = 0
    for x in range(0,len(tbl_list)//13):
        col_name = tbl_list[x+idx]
        list_vals = []
        for val in range(x+1,x+13):
            list_vals.append(tbl_list[idx+val])
        df[col_name]=list_vals
        idx = idx+12
    return df

new df called monthly_sales_df

In [3]:
url = 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/' 
df_sales = get_monthly_sales_df(url, '2019', 2)
url = 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
df = get_monthly_sales_df(url, '2020', 1)
monthly_sales_df = pd.concat((df_sales, df))
#monthly_sales_df.columns

pickle df of monthly sales for 2019 and 2020 to data folder

In [ ]:
monthly_sales_df.to_pickle('../data/monthly_sales_df.pkl')

#### Get All Model Yearly Sales for 2005-2020

In [4]:
def get_model_sales_df(url, year):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #read url page into list of pandas dfs 
    response = requests.get(url, headers=headers)
    page = response.text
    df_list = pd.read_html(page)
    
    #find correct df based on number of data frames on url page and 
    if len(df_list) == 1 or year == '2019':
        df = pd.DataFrame(df_list[0])
    elif len(df_list) == 2:
        df = pd.DataFrame(df_list[1])
    else:
        df = pd.DataFrame(df_list[len(df_list)-1])
    
    #special case for 2020 data because sales data is in monthly sales format for each model
    if year == '2020':  
        soup = BeautifulSoup(page, "lxml")
    
        #find car sales data and turn it into a list
        tbl_str = soup.find_all('tbody')[1].text
        tbl_list = tbl_str.split('\n')
        tbl_list = [i for i in tbl_list if i] 
        
        columns = ('Model', 'Year', 'Total_Sales')
        df = pd.DataFrame(columns=columns)

        #fill empty df with monthly sales for all makes and models
        model_name = ''
        idx = 0
        index = 0
        for x in range(0,len(tbl_list)//13):
            model_name = tbl_list[x+idx]
            year_sum = 0
            for val in range(x+1,x+13):
                month_total = tbl_list[idx+val].replace(',','')
                month_total = int(month_total)
                year_sum = year_sum + month_total
            #df1 = pd.DataFrame([model_name, year_sum], columns = ['Model', year])
            df = df.append(pd.DataFrame({'Model': model_name, 'Year': year, 'Total_Sales': year_sum}, index=[index]), 
                           ignore_index=True)
            index = index+1
            #df.append(df1)
            idx = idx+12
        return df
    
    if year == '2012':
        df = df.iloc[:, [2,3]]
    elif year == '2005':
        df = df.iloc[:, [1,3]]
    elif year in ['2017','2018', '2019', '2020']:
        df = df.iloc[:, [0,4]]
    else:
        df = df.iloc[:, [1,2]]
    df.columns = ['Model', 'Total_Sales']
    df['Year'] = year
    
    #* indicate further breakdown of sum totals, overall totals be removed further down when duplicates are removed
    #other symbols refer to subnotes in the tables and are not apart of model names
    symbols = ['*', '²', '¹', '^', '†', '‡']
    
    for s in symbols:
        df['Model'] = df['Model'].str.replace(s,'')

    df.dropna(subset=['Model'], inplace=True)
    
    df['Model'] = df['Model'].str.rstrip()
    df['Model'] = df['Model'].str.lstrip()
    
    #remove first in set of duplicates b/c first is a sum of a car and the hybrid model
    
    df.drop_duplicates(subset='Model', keep='last', inplace=True)
    
    df['Total_Sales'] = df['Total_Sales'].apply(pd.to_numeric, errors='coerce')
    
    return df

In [5]:
url = 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/'
yearly_sales_df= get_model_sales_df(url = url, year = '2005')

<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')


In [6]:
url_dict = {'2006': 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/',
            '2007': 'https://www.goodcarbadcar.net/usa-2007-vehicle-sales-rankings-by-mode/',
            '2008': 'https://www.goodcarbadcar.net/2008-america-auto-sales-rankings-by-mode/',
            '2009': 'https://www.goodcarbadcar.net/usa-auto-sales-rankings-by-model-2009/',
            '2010': 'https://www.goodcarbadcar.net/2010-america-auto-sales-rankings-by-mode/',
            '2011': 'https://www.goodcarbadcar.net/top-268-best-selling-vehicles-2011-year/',
            '2012': 'https://www.goodcarbadcar.net/2012-usa-auto-sales-rankings-by-model7/',
            '2013': 'https://www.goodcarbadcar.net/usa-vehicle-sales-rankings-by-model-december-2013-year-end/',
            '2014': 'https://www.goodcarbadcar.net/usa-all-cars-sales-figures-2014-december-year-end/',
            '2015': 'https://www.goodcarbadcar.net/usa-car-sales-by-model-2015-year-end-december/',
            '2016': 'https://www.goodcarbadcar.net/usa-2016-vehicle-sales-by-model-manufacturer-brand/',
            '2017': 'https://www.goodcarbadcar.net/december-2017-year-end-u-s-passenger-car-sales-rankings-top-171-best-selling-cars-america-every-car-ranked/',
            '2018': 'https://www.goodcarbadcar.net/december-2018-the-best-selling-vehicles-in-america-every-vehicle-ranked/',
            '2019': 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/',
            '2020': 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
           }

years = ['2006']
for key, value in url_dict.items():
    #print(key)
    years.append(key)
    df= get_model_sales_df(url = value, year = key)
    #print(df)
    yearly_sales_df = pd.concat([yearly_sales_df, df], axis=0)
    #yearly_sales_df = pd.merge(yearly_sales_df, df, on='Model', how='outer')
    #print(yearly_sales_df[yearly_sales_df.Model.str.contains("Toyota")])

print(yearly_sales_df)

<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single chara

                     Model Total_Sales  Year
0            Ford F-Series    901463.0  2005
1      Chevrolet Silverado    705980.0  2005
2             Toyota Camry    431703.0  2005
3    Toyota Corolla/Matrix    341290.0  2005
4                Dodge Ram    400543.0  2005
..                     ...         ...   ...
298        Volvo 60-Series       15729  2020
299        Volvo 90-Series        3195  2020
300             Volvo XC40       23778  2020
301             Volvo XC60       32078  2020
302             Volvo XC90       34251  2020

[4518 rows x 3 columns]


In [7]:
# columns = ('Model','Total_Sales')
# total_sales_df = pd.DataFrame(columns=columns)
# total_sales_df = total_sales_df.fillna(0)
# total_sales_df['Model'] = yearly_sales_df['Model']

# total_sales_df['Total_Sales'] = yearly_sales_df[years].sum(axis=1)
# #print(total_sales_df)

In [8]:
# total_sales_df.sort_values('Total_Sales', ascending=False, inplace=True)
yearly_sales_df = yearly_sales_df[yearly_sales_df.Total_Sales != 0]

remove_strings = ["Market",'Total','Family','Brand','Passenger Cars, SUVs, Crossovers','Minivans','Pickup Trucks',
                 'Commercial Vans', 'COMPANY', 'MOTOR', 'GROUP', 'AMERICAN', 'AUTOMOBILES', 'JAGUAR', 'DAIMLER']

for string in remove_strings:
    yearly_sales_df = yearly_sales_df[~yearly_sales_df.Model.str.contains(string)]

#total_sales_df = total_sales_df[total_sales_df.Model != '*Market*']
yearly_sales_df = yearly_sales_df.reset_index(drop=True)
yearly_sales_df

,Model,Total_Sales,Year
0,Ford F-Series,901463.0,2005
1,Chevrolet Silverado,705980.0,2005
2,Toyota Camry,431703.0,2005
3,Toyota Corolla/Matrix,341290.0,2005
4,Dodge Ram,400543.0,2005
...,...,...,...
4388,Volvo 60-Series,15729,2020
4389,Volvo 90-Series,3195,2020
4390,Volvo XC40,23778,2020
4391,Volvo XC60,32078,2020


In [9]:
def get_brand_links(org_url):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #org_url = 'https://www.carspecs.us/'
    response = requests.get(org_url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #tbl = soup.find_all(class_='modelnamesandmakes_item')
    #link_list = soup.find_all(id_ = 'homepage-browsemakes')
    brand_link_dict = dict()
    link_list = soup.find_all('li')
    for x in link_list[7:]:
        brand = x.text
        link = 'https://www.carspecs.us' + x.find('a')['href']
        brand_link_dict[brand] = link
        time.sleep(0.5)
    return brand_link_dict

In [18]:
def get_model_links(brand, url):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #load url page with all car models for specified brand url
    response = requests.get(url, headers=headers) #random user agent
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find first div with all model links
    div_list = soup.find_all('div', class_='pure-u-1 pure-u-md-1-2')
    
    #new df to store all makes and models for all years past 2004
    columns = ('Model', 'Year', 'Model_url')
    df = pd.DataFrame(columns=columns)
    #model_link_dict = dict()
    
    #loop through all model links for specified brand 
    for x in div_list[1].find_all('li'):
        model = brand + ' ' + x.text 
        link = org_url + x.find('a')['href'] #model url
        
        #load model url to get list of model year urls
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        year_links = soup.find_all('li') #list of all year urls for specified model
        
        #print(year_links).text
        #loop through all year links starting at index 7
        idx = 7
        index = 0
        for y in year_links[7:]:
            year = year_links[idx].text
            if year in years: #only add year links that are 2005 to 2020
                #print(year)
                model_link = 'https://www.carspecs.us' + year_links[idx].find('a')['href']
                df = df.append(pd.DataFrame({'Model': model, 'Year': year, 'Model_url': model_link}, index=[index]), 
                                ignore_index=True)
                index = index+1
                #model_link_dict[model] = (year, model_link)
                idx = idx+1
        time.sleep(0.5) #pause 
            
    return df
#    return model_link_dict

In [25]:
def get_model_specs(url):
#url = 'https://www.carspecs.us//cars/2020/acura/mdx'
    response = requests.get(url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    doors =passengers =speed =horsepower =drive =engine =tank =volume =length =width =height = float('NaN')
    mpg = ''
    
    try:
        price = soup.find(text=re.compile('starting from'))
        price = price.findNext().text
    
    except:
        price = float('NaN')
    
    div_list = soup.find('div', class_='car-details').find_all('div')
    #print(div_list)

    for div in div_list[1:]:
        spec_list = div.text.split('\n')
        spec_list = [i for i in spec_list if i]
        #print(spec_list)
        if spec_list:
            if 'RPM' not in spec_list[0]:
                if 'Passenger Doors' in spec_list[0]:
                    doors = int(spec_list[-1])
                if 'Passenger Capacity' in spec_list[0]:
                    passengers = int(spec_list[-1])
                if 'mph' in spec_list[0]:
                    speed = spec_list[-1]
                if 'Horsepower' in spec_list[0]:
                    horsepower = spec_list[-1]
                if 'Drive type' in spec_list[0]:
                    drive = spec_list[-1]
                if 'combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Engine type' in spec_list[0]:
                    engine = spec_list[-1]
                    engine = engine.replace('\t', '')
                if 'tank capacity' in spec_list[0]:
                    tank = spec_list[-1]
                if 'EPA interior' in spec_list[0]:
                    volume = spec_list[-1]
                if 'Length' in spec_list[0]:
                    length = spec_list[-1]
                if 'Width' in spec_list[0]:
                    width = spec_list[-1]
                if 'Height' in spec_list[0]:
                    height = spec_list[-1]
        
        if mpg == '':
            mpg = soup.find(text=re.compile('highway mpg'))
            try:
                mpg_list = mpg.split('/ ')
                city = mpg_list[0].lstrip('\r\n ')
                highway = mpg_list[1]
                mpg = (int(city[0:2])+int(highway[0:2]))/2
            except:
                mpg = float('NaN')
                
    time.sleep(.1+.5*random.random()) #random pause 
    
    return [price, doors, passengers, speed, horsepower, drive, mpg, engine, tank, volume, length, width, height]

#### Get a dictionary of links to all models of a car brand 

In [12]:
org_url = 'https://www.carspecs.us/'
brand_links = get_brand_links(org_url)
#print(brand_links)


#### Get a dictionary of links to all years of a specific make and model 

In [20]:
# all_model_links = dict()
#for key, value in brand_links.items():
#     #print(get_model_links(key, brand_links[key]))
#     all_model_links.update(get_model_links(key, brand_links[key]))
#     #print(all_model_links)
# #rint(all_model_links)
columns = ('Model', 'Year', 'Model_url')
model_links_df = pd.DataFrame(columns=columns)
for key, value in brand_links.items():
    #print(get_model_links(key, brand_links[key]))
    df = get_model_links(key, brand_links[key])
    model_links_df = model_links_df.append(df, ignore_index=True)
    #model_links_df = pd.concat(model_links_df,df)
    #print(model_links_df)


In [21]:
print(model_links_df)

           Model  Year                                     Model_url
0      Acura ILX  2020   https://www.carspecs.us/cars/2020/acura/ilx
1      Acura ILX  2019   https://www.carspecs.us/cars/2019/acura/ilx
2      Acura ILX  2018   https://www.carspecs.us/cars/2018/acura/ilx
3      Acura ILX  2017   https://www.carspecs.us/cars/2017/acura/ilx
4      Acura ILX  2016   https://www.carspecs.us/cars/2016/acura/ilx
...          ...   ...                                           ...
6576  Volvo XC90  2010  https://www.carspecs.us/cars/2010/volvo/xc90
6577  Volvo XC90  2009  https://www.carspecs.us/cars/2009/volvo/xc90
6578  Volvo XC90  2008  https://www.carspecs.us/cars/2008/volvo/xc90
6579  Volvo XC90  2007  https://www.carspecs.us/cars/2007/volvo/xc90
6580  Volvo XC90  2006  https://www.carspecs.us/cars/2006/volvo/xc90

[6581 rows x 3 columns]


#### Create new df containging specs of every make, model, and year of a car

In [ ]:
columns = ('Model', 'url', 'price', 'doors', 'passengers', 'speed_sec', 'horsepower_hp', 'drive', 'mpg', 'engine', 
           'tank_gal', 'volume_cuft', 'length_in', 'width_in', 'height_in')
model_spec_df = pd.DataFrame(columns=columns) 
idx = 0
for index, row in model_links_df.iterrows():
    #print(row[2])
    specs = get_model_specs(row[2])
    #print(specs)
#     print(value[0])
#     print(value[1])
    model_spec_df = model_spec_df.append(pd.DataFrame({'Model': row[0],'Year':row[1], 'url': row[2], 'price': specs[0],'doors': specs[1],
                                                      'passengers': specs[2],'speed_sec': specs[3], 'horsepower_hp': specs[4],
                                                       'drive': specs[5],'mpg': specs[6],'engine': specs[7],
                                                       'tank_gal': specs[8], 'volume_cuft': specs[9], 'length_in': specs[10],
                                                      'width_in': specs[11],'height_in': specs[12]}, index=[idx]), ignore_index=True)
    idx = idx +1
    #print(model_spec_df)

https://www.carspecs.us/cars/2020/acura/ilx
['$25,900', 4, 5, '6.63 sec', '201 hp', 'Front Wheel Drive ', '28 mpg', 'Gas', '13.20 gal.', '89.30 cu.ft.', '182.20 in.', '70.60 ', '55.60 in.']
https://www.carspecs.us/cars/2019/acura/ilx
['$25,900', 4, 5, '6.63 sec', '201 hp', 'Front Wheel Drive ', '28 mpg', 'Gas', '13.20 gal.', '89.30 cu.ft.', '182.20 in.', '70.60 ', '55.60 in.']
https://www.carspecs.us/cars/2018/acura/ilx
[nan, 4, 5, '6.63 sec', '201 hp', 'Front Wheel Drive ', '29 mpg', 'Gas', '13.20 gal.', '89.30 cu.ft.', '181.90 in.', '70.60 ', '55.60 in.']
https://www.carspecs.us/cars/2017/acura/ilx
[nan, 4, 5, '6.63 sec', '201 hp', 'Front Wheel Drive ', '29 mpg', 'Gas', '13.20 gal.', '89.30 cu.ft.', '181.90 in.', '70.60 ', '55.60 in.']
https://www.carspecs.us/cars/2016/acura/ilx
['$23,652', 4, 5, '6.63 sec', '201 hp', 'Front Wheel Drive ', '29 mpg', 'Gas', '13.20 gal.', nan, '181.90 in.', '70.60 ', '55.60 in.']
https://www.carspecs.us/cars/2015/acura/ilx
['$18,664', nan, nan, '9.09 s

[nan, nan, nan, '6.10 sec', '300 hp', ' All wheel drive  ', 19.0, 'Gas', '19.40 gal.', '112.20 cu.ft.', '195.80 in.', '72.70 ', '57.20 in.']
https://www.carspecs.us/cars/2008/acura/rl
['$10,479', 4, 5, '6.18 sec', '290 hp', 'All Wheel Drive ', 20.0, 'Gas', '19.40 gal.', nan, '193.60 in.', '72.70 ', '57.10 in.']
https://www.carspecs.us/cars/2007/acura/rl
[nan, nan, nan, '6.17 sec', '290 hp', ' All wheel drive  ', 20.0, 'Gas', '19.40 gal.', '112.20 cu.ft.', '193.60 in.', '72.70 ', '57.10 in.']
https://www.carspecs.us/cars/2006/acura/rl
['$7,770', 4, 5, '6.17 sec', '290 hp', 'All Wheel Drive ', 22.0, 'Gas', '19.40 gal.', nan, '193.60 in.', '72.70 ', '57.10 in.']
https://www.carspecs.us/cars/2020/acura/rlx
['$54,900', 4, 5, '6.59 sec', '310 hp', 'Front Wheel Drive ', '23 mpg', 'Gas', '18.50 gal.', '102.10 cu.ft.', '198.10 in.', '74.40 ', '57.70 in.']
https://www.carspecs.us/cars/2019/acura/rlx
['$61,900', 4, 5, '4.75 sec', '377 hp', 'All Wheel Drive ', '28 mpg', 'Gas', '15.10 gal.', '102.1

['$67,150', 2, 2, '4.87 sec', '237 hp', 'Rear Wheel Drive ', '28 mpg', 'Gas', '10.60 gal.', nan, '157.00 in.', '73.50 ', '46.60 in.']
https://www.carspecs.us/cars/2018/alfa-romeo/4c
[nan, 2, 2, '4.87 sec', '237 hp', 'Rear Wheel Drive ', '28 mpg', 'Gas', '10.60 gal.', nan, '157.00 in.', '73.50 ', '46.60 in.']
https://www.carspecs.us/cars/2017/alfa-romeo/4c
[nan, 2, 2, nan, '237 hp', 'Rear Wheel Drive ', '28 mpg', 'Gas', '10.50 gal.', nan, '157.00 in.', '73.50 ', '46.60 in.']
https://www.carspecs.us/cars/2016/alfa-romeo/4c
['$53,379', 2, 2, nan, '237 hp', 'Rear Wheel Drive ', '28 mpg', 'Gas', '10.50 gal.', nan, '157.00 in.', '73.50 ', '46.60 in.']
https://www.carspecs.us/cars/2015/alfa-romeo/4c
['$37,191', nan, nan, '4.57 sec', '237 hp', ' Rear wheel drive  ', 29.0, 'Gas', '10.50 gal.', nan, '157.00 in.', nan, '46.60 in.']
https://www.carspecs.us/cars/2018/alfa-romeo/4c-spider
['$65,900', 2, 2, '4.87 sec', '237 hp', 'Rear Wheel Drive ', 0, 'Gas', '10.50 gal.', nan, '157.00 in.', '73.50 '

['$114,985', nan, nan, '4.26 sec', '550 hp', ' Rear wheel drive  ', 16.0, 'Gas', '23.90 gal.', '97.00 cu.ft.', '197.60 in.', '75.90 ', '53.10 in.']
https://www.carspecs.us/cars/2006/aston-martin/v12-vanquish
['$41,600', nan, nan, '3.73 sec', '520 hp', ' Rear wheel drive  ', 13.0, 'Gas', '22.00 gal.', nan, '183.70 in.', '75.70 ', '51.90 in.']
https://www.carspecs.us/cars/2016/aston-martin/v12-vantage
['$150,898', 2, 2, '3.65 sec', '565 hp', 'Rear Wheel Drive ', '14 mpg', 'Gas', '21.10 gal.', nan, '172.60 in.', '73.50 ', '49.20 in.']
https://www.carspecs.us/cars/2015/aston-martin/v12-vantage
['$135,581', 2, 2, '3.79 sec', '565 hp', 'Rear Wheel Drive ', 0, 'Gas', '21.10 gal.', nan, '172.60 in.', '73.40 ', '49.60 in.']
https://www.carspecs.us/cars/2013/aston-martin/v12-vantage
['$88,494', 2, 2, '3.70 sec', '510 hp', 'Rear Wheel Drive ', 0, 'Gas', '21.10 gal.', nan, '172.60 in.', '73.40 ', '48.60 in.']
https://www.carspecs.us/cars/2012/aston-martin/v12-vantage
[nan, nan, nan, '3.70 sec', '5

[nan, 4, 5, nan, '292 hp', 'All Wheel Drive ', '25 mpg', 'Gas', '14.50 gal.', nan, '175.80 in.', '70.70 ', '54.80 in.']
https://www.carspecs.us/cars/2017/audi/a3
[nan, 4, 5, '8.40 sec', '204 hp', 'Front Wheel Drive ', '34 mpg', 'Gas', '10.60 gal.', nan, '169.70 in.', '70.30 ', '56.10 in.']
https://www.carspecs.us/cars/2016/audi/a3
['$29,646', 2, 4, nan, '170 hp', 'Front Wheel Drive ', '28 mpg', 'Gas', '13.20 gal.', nan, '175.00 in.', '7.50 in.', '55.50 in.']
https://www.carspecs.us/cars/2015/audi/a3
['$20,631', 4, 5, nan, '170 hp', 'Front Wheel Drive ', '27 mpg', 'Gas', '13.20 gal.', nan, '175.00 in.', '70.70 ', '55.80 in.']
https://www.carspecs.us/cars/2013/audi/a3
['$14,087', 4, 5, '6.98 sec', '200 hp', 'Front Wheel Drive ', '24 mpg', 'Gas', '14.50 gal.', nan, '169.00 in.', '78.50 ', '56.00 in.']
https://www.carspecs.us/cars/2012/audi/a3
['$12,100', 4, 5, '9.17 sec', '140 hp', 'Front Wheel Drive ', '34 mpg', 'Gas', '14.60 gal.', nan, '169.00 in.', '78.50 ', '56.00 in.']
https://www.c

...

['$11,818', nan, nan, '8.85 sec', '244 hp', ' Front wheel drive  ', 21.0, 'Gas', '21.00 gal.', nan, '202.10 in.', '77.10 ', '70.00 in.']
https://www.carspecs.us/cars/2009/honda/odyssey
['$9,238', 4, 8, '8.75 sec', '244 hp', 'Front Wheel Drive ', '20 mpg', 'Gas', '21.00 gal.', nan, '202.10 in.', '77.10 ', '70.00 in.']
https://www.carspecs.us/cars/2008/honda/odyssey
['$5,430', 4, 7, '8.48 sec', '244 hp', 'Front Wheel Drive ', 19.5, 'Gas', '21.00 gal.', nan, '202.10 in.', '77.10 ', '68.80 in.']
https://www.carspecs.us/cars/2007/honda/odyssey
['$6,641', nan, nan, '8.90 sec', '244 hp', ' Front wheel drive  ', 20.5, 'Gas', '21.00 gal.', nan, '201.00 in.', '77.10 ', '70.00 in.']
https://www.carspecs.us/cars/2006/honda/odyssey
['$5,183', nan, nan, '8.61 sec', '244 hp', ' Front wheel drive  ', 20.0, 'Gas', '21.00 gal.', nan, '201.00 in.', '77.10 ', '70.00 in.']
https://www.carspecs.us/cars/2020/honda/passport
['$31,990', 4, 5, '7.09 sec', '280 hp', 'Front Wheel Drive ', '22 mpg', 'Gas', '19.50 

[nan, nan, nan, nan, '393 hp', ' Four wheel drive  ', 0, 'Flex Fuel', '32.00 gal.', nan, '189.80 in.', '81.20 ', '79.20 in.']
https://www.carspecs.us/cars/2008/hummer/h2-sut
[nan, nan, nan, nan, '393 hp', ' Four wheel drive  ', 0, 'Gas', '32.00 gal.', nan, '189.80 in.', '81.20 ', '79.20 in.']
https://www.carspecs.us/cars/2007/hummer/h2-sut
['$9,740', nan, nan, '8.04 sec', '325 hp', ' Four wheel drive  ', 0, 'Gas', '32.00 gal.', nan, '203.60 in.', '81.20 ', '79.20 in.']
https://www.carspecs.us/cars/2006/hummer/h2-sut
[nan, nan, nan, nan, '325 hp', ' Four wheel drive  ', 0, 'Gas', '32.00 gal.', nan, '203.60 in.', '81.20 ', '79.20 in.']
https://www.carspecs.us/cars/2009/hummer/h2-suv
[nan, nan, nan, '6.28 sec', '393 hp', 'full-time four-wheel ', 0, nan, '32.00 gal.', nan, nan, nan, nan]
https://www.carspecs.us/cars/2008/hummer/h2-suv
['$12,068', nan, nan, '6.28 sec', '393 hp', 'full-time four-wheel ', 0, nan, '32.00 gal.', nan, nan, nan, nan]
https://www.carspecs.us/cars/2007/hummer/h2-su

['$3,893', 4, 5, '6.96 sec', '263 hp', 'Front Wheel Drive ', 22.5, 'Gas', '19.80 gal.', nan, '192.70 in.', '72.80 ', '58.70 in.']
https://www.carspecs.us/cars/2020/hyundai/elantra
['$21,600', 4, 5, '8.76 sec', '128 hp', 'Front Wheel Drive ', '36 mpg', 'Gas', '14.00 gal.', '95.80 cu.ft.', '181.90 in.', '70.90 ', '56.50 in.']
https://www.carspecs.us/cars/2019/hyundai/elantra
['$20,450', 4, 5, '8.59 sec', '161 hp', 'Front Wheel Drive ', '27 mpg', 'Gas', '14.00 gal.', nan, '170.90 in.', '70.70 ', '57.70 in.']
https://www.carspecs.us/cars/2018/hyundai/elantra
[nan, 4, 5, '8.89 sec', '147 hp', 'Front Wheel Drive ', '33 mpg', 'Gas', '14.00 gal.', '95.80 cu.ft.', '179.90 in.', '70.90 ', '56.50 in.']
https://www.carspecs.us/cars/2017/hyundai/elantra
[nan, 4, 5, '7.40 sec', '173 hp', 'Front Wheel Drive ', '27 mpg', 'Gas', '13.20 gal.', '96.10 cu.ft.', '169.30 in.', '70.10 ', '57.90 in.']
https://www.carspecs.us/cars/2016/hyundai/elantra
['$16,038', 4, 5, '8.06 sec', '173 hp', 'Front Wheel Drive 

['$8,062', nan, nan, '6.12 sec', '290 hp', ' Rear wheel drive  ', 22.5, 'Gas', '19.30 gal.', '125.30 cu.ft.', '195.90 in.', '73.40 ', '58.30 in.']
https://www.carspecs.us/cars/2016/hyundai/genesis-coupe
['$28,309', 2, 4, '5.07 sec', '348 hp', 'Rear Wheel Drive ', '19 mpg', 'Gas', '17.20 gal.', nan, '182.30 in.', '73.40 ', '54.50 in.']
https://www.carspecs.us/cars/2015/hyundai/genesis-coupe
['$18,457', nan, nan, '4.74 sec', '348 hp', ' Rear wheel drive  ', 20.0, 'Gas', '17.20 gal.', '98.80 cu.ft.', '182.30 in.', '73.40 ', '54.50 in.']
https://www.carspecs.us/cars/2014/hyundai/genesis-coupe
['$16,994', 2, 4, '6.06 sec', '274 hp', 'Rear Wheel Drive ', '21 mpg', 'Gas', '17.20 gal.', nan, '182.30 in.', '73.40 ', '54.50 in.']
https://www.carspecs.us/cars/2013/hyundai/genesis-coupe
['$11,882', nan, nan, '5.05 sec', '348 hp', ' Rear wheel drive  ', 20.5, 'Gas', '17.20 gal.', '98.80 cu.ft.', '182.30 in.', '73.40 ', '54.50 in.']
https://www.carspecs.us/cars/2012/hyundai/genesis-coupe
['$12,300',

['$32,900', 4, 5, '6.48 sec', '240 hp', 'All Wheel Drive ', '22 mpg', 'Gas', '17.40 gal.', nan, '185.00 in.', '74.00 ', '66.10 in.']
https://www.carspecs.us/cars/2017/hyundai/santa-fe-sport
['$33,450', 4, 5, '6.48 sec', '240 hp', 'All Wheel Drive ', '22 mpg', 'Gas', '17.40 gal.', nan, '185.00 in.', '74.00 ', '66.10 in.']
https://www.carspecs.us/cars/2016/hyundai/santa-fe-sport
['$20,209', 4, 5, '7.97 sec', '190 hp', 'Front Wheel Drive ', '23 mpg', 'Gas', '17.40 gal.', nan, '184.60 in.', '74.00 ', '66.10 in.']
https://www.carspecs.us/cars/2015/hyundai/santa-fe-sport
['$17,215', 4, 5, '7.97 sec', '190 hp', 'Front Wheel Drive ', '23 mpg', 'Gas', '17.40 gal.', nan, '184.60 in.', '74.00 ', '66.10 in.']
https://www.carspecs.us/cars/2013/hyundai/santa-fe-sport
[nan, nan, nan, '8.56 sec', '190 hp', 'front-wheel ', 25.0, nan, '17.40 gal.', nan, nan, nan, nan]
https://www.carspecs.us/cars/2019/hyundai/santa-fe-xl
['$41,300', 4, 6, '5.91 sec', '290 hp', 'All Wheel Drive ', 0, 'Gas', '18.80 gal.',

#### Left merge Sales df to specs df and find info

perform intial merge

In [ ]:
model_spec_sales_df = yearly_sales_df.merge(model_spec_df, how = 'left',on=["Model",'Year'])
model_spec_sales_df.info()
model_spec_sales_df.head()

We see that some of the cars with the top sales did not get any spec data. This needs to be further investigated to see if 

drop volume_cuft from model_specs_df because not enough data points and cant be easily estimated

In [ ]:
#model_spec_df.drop(labels=['volume_cuft'], axis=1, inplace=True)

Find columns in specs df that did not merge

In [ ]:
unique_sales_df = (yearly_sales_df.merge(model_spec_df, on='Model', how='outer', indicator=True)
            .query('_merge != "both"').drop(columns='_merge'))
unique_sales_df['Model'].value_counts().head(20)
#unique_sales_df.groupby('Model').sum()

Identify mismatches with largest value counts and replace strings

In [ ]:
ford_f_models = model_spec_df[model_spec_df['Model'].str.contains('Ford F')] #.dropna()
print (ford_f_models)
model_spec_df['Model'] = model_spec_df['Model'].str.replace('150','Series')

In [ ]:
chevy_models = model_spec_df[model_spec_df['Model'].str.contains('Chevrolet S')].dropna()
print (chevy_models)
model_spec_df['Model'] = model_spec_df['Model'].str.replace('150','Series')

Remove symbols and units from data in df so data can be changed to numeric datatype

In [ ]:
replace_list = ['hp', 'mpg', 'gal.', 'cu.ft.', 'in.', ',', 'sec', '$', '$']
for s in replace_list:
    model_spec_sales_df = model_spec_sales_df.replace(s,'', regex=True) 
model_spec_sales_df['price'] = model_spec_sales_df['price'].str.replace('$','')

Change all columns to numeric except for drive, engine, doors, and passengers

In [ ]:
num_cols = ['price',  'speed_sec', 'horsepower_hp', 'mpg', 'tank_gal', 'volume_cuft', 
            'width_in', 'length_in','height_in']
for col in num_cols:
    model_spec_sales_df[col] = model_spec_sales_df[col].apply (pd.to_numeric, errors='coerce')
model_spec_sales_df['doors'] = model_spec_sales_df['doors'].astype(str)
model_spec_sales_df['passengers'] = model_spec_sales_df['passengers'].astype(str)
model_spec_sales_df,info()

fill in na values with existing model data from another year

In [ ]:
all_model_specs = model_spec_df.copy() 
all_model_specs['passengers'] = all_model_specs['passengers'].astype(int)
all_model_specs['doors'] = all_model_specs['doors'].astype(int)
all_model_specs_dict = dict(list(all_model_specs.groupby('Model').mean()))
all_model_specs_dict

In [ ]:
for index, row in model_spec_sales_df.isnull().values.any(axis=1).iterrows():
    specs = all_model_specs_dict(row['Model'])
    idx = 0
    for x in row:
        if x is None and specs[idx]:
            model_spec_sales_df.iloc[index,idx] = specs[idx]
model_spec_sales_df.info()

drop uneeded columns and remove rows with na values

In [ ]:
model_spec_sales_df.drop(labels=['Model', 'Year_y', 'url'], axis=1, inplace=True)
# model_spec_sales_df.dropna(how = ['price', 'doors', 'passengers', 'speed_sec', 'horsepower_hp', 'drive', 'mpg', 
#                                   'engine', 'tank_gal', 'volume_cuft', 'length_in', 'width_in', 'height_in'], inplace=True)
model_spec_sales_df.dropna(inplace=True)
model_spec_sales_df.info()

Clean drive feature

In [ ]:
# model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.lower()
# model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.lstrip()
# model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.rstrip()
#model_spec_sales_df = model_spec_sales_df.dropna()
model_spec_sales_df['drive'].value_counts()

In [ ]:
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('rear-wheel','rear wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('front-wheel','front wheel drive')
#replace = ['full-time all wheel', 'attesa e-ts full-time all wheel', 'automatic full-time all wheel']

model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('automatic full-time all wheel',
                                                                        'all wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('versatrak ','')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('full-time all wheel','all wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('real time automatic full-time four-wheel',
                                                                        'all wheel drive')
# for s in replace:
#     model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace(s,'all wheel drive')
model_spec_sales_df['drive'].value_counts()

In [ ]:
model_spec_sales_df.info()
model_spec_sales_df.describe()

Remove outliers, more than 3 std dev from mean

In [ ]:
z = np.abs(stats.zscore(model_spec_sales_df['price', 'speed_sec', 'horsepower_hp', 'mpg', 'tank_gal', 
                                            'length_in', 'width_in', 'height_in']))
model_spec_sales_df_o = model_spec_sales_df[(z < 3).all(axis=1)]

In [ ]:
model_spec_sales_df.info()
model_spec_sales_df.sample(10)

pickle model sales and specs df to data folder

In [ ]:
model_spec_sales_df.to_pickle('../data/model_spec_sales_df.pkl')